In [0]:
!curl https://colab.chainer.org/install | sh -
!apt-get install graphviz
!pip install 'chaineripy'
!pip install 'chainercv'

import sys
import os
from functools import partial
import json


from chainer.links.caffe.caffe_function import CaffeFunction
from chainer.serializers import npz
from chainer.datasets import cifar, split_dataset_random, TransformDataset
from chainer import links as L
from chainer import initializers, iterators
from chainer.trainer import extensions
from google.colab import drive
import numpy as np


base_dir = './drive/My Drive/study'
sys.path.append(base_dir)
drive.mount('/content/drive')

caffemodel = CaffeFunction(base_dir + '/squeezenet/caffe/squeezenet_v1.1.caffemodel')
npz.save_npz('squeezenet_v1_1.npz', caffemodel, compression=False)
caffemodel = CaffeFunction(base_dir + '/squeezenet/caffe/squeezeNet_residual.caffemodel')
npz.save_npz('squeeze_resnet.npz', caffemodel, compression=False)

from squeezenet.networks.squeezenet import SqueezeNet
from squeezenet.networks.squeezenet_bn import SqueezeNetBN
from squeezenet.networks.squeeze_resnet import SqueezeResNet
from squeezenet.networks.squeeze_pre_resnet import SqueezePreResNet
from execute import run_train
from transform import transform

# Dataset setup
train_val, test = cifar.get_cifar10(scale=255.)
train_size      = int(len(train_val) * 0.9)
train, valid    = split_dataset_random(train_val, train_size, seed=0)

mean = np.mean([x for x, _ in train], axis=(0, 2, 3))
std  = np.std([x for x, _ in train], axis=(0, 2, 3))

params = json.load(open(base_dir + '/params.json', "r"))

In [0]:
# Data Augmentation
transform_train = partial(transform, mean=mean, std=std, train=True, **params)
transform_valid = partial(transform, mean=mean, std=std, **params)

processed_train = TransformDataset(train, transform_train)
processed_valid = TransformDataset(valid, transform_valid)
train_iter = iterators.SerialIterator(train, params['batchsize'])
valid_iter = iterators.SerialIterator(valid, params['batchsize'], repeat=False, shuffle=False)

# Model
model = SqueezeNet(10, pretrained_model='squeezenet_v1_1.npz', init_param=initializers.HeNormal())
net = L.Classifier(model)

# Evaluator
evaluator = extensions.Evaluator(valid_iter, net, device=params['gpu_id'])

# RUN!
params['batchsize']     = 128
params['changed_layer'] = [net.predictor.conv10.W, net.predictor.conv10.b]
params['freeze_layer']  = ['base/conv1', 'base/fire2', 'base/fire3', 'base/fire4', 'base/fire5']
params['result_dir']    = 'drive/My Drive/study/result'

run_train(train_iter, net, evaluator **params)

/usr/local/lib/python3.6/dist-packages/chainer/optimizer.py:524: UserWarning: Hook timing attribute not in ('pre', 'post'), defaulting timing to 'pre'.
  warnings.warn("Hook timing attribute not in ('pre', 'post'), "


epoch       main/loss   main/accuracy  val/main/loss  val/main/accuracy  lr          elapsed_time
1           1.8634      0.322754       1.20593        0.5625             0.001       172.701       
2           1.17388     0.578125       0.994086       0.650391           0.001       346.452       
3           0.939049    0.66729        0.718133       0.754492           0.001       519.857       
4           0.805166    0.718861       0.622207       0.788086           0.001       693.688       
5           0.716353    0.748976       0.586388       0.799414           0.001       867.062       
6           0.663403    0.769487       0.551529       0.806641           0.001       1041.77       
7           0.608284    0.78795        0.499005       0.819727           0.001       1214.95       
8           0.571525    0.801025       0.467132       0.841602           0.001       1388.49       
9           0.541252    0.812145       0.425097       0.855469           0.001       1562.15       
10